# Getting started

A lot of people get diagnosed with squamous cell carcinoma in the head & neck region ([HNSCC](https://en.wikipedia.org/wiki/Head_and_neck_cancer)), which frequently metastasizes via the lymphatic system. We set out to develop a methodology to predict the risk of a new patient having metastases in so-called lymph node levels (LNLs), based on their personal diagnose (e.g. findings from a CT scan) and information of previously diagnosed and treated patients. And that's exactly what this code enables you to do as well.

As mentioned, this package is meant to be a relatively simple-to-use frontend. The math is done under the hood and one does not need to worry about it a lot. But let's have a quick look at what we're doing here.

We will assume that you have already read the section on how to install the module and followed its instructions.

## Importing

First, let's import the package.

In [ ]:
import lymph

## Graph

The model is based on the assumption that one can represent the lymphatic system as a directed graph. The arcs in that graph represent the direction of the lymphatic flow and therefore also the direction of metastatic spread. Hence, the first thing to do is to define a graph that represents the drainage pathways of the lymphatic system aptly.

Here, this is done via a dictionary:

In [ ]:
graph_dict = {
    ('tumor', 'T')  : ['I', 'II', 'III', 'IV'], 
    ('lnl'  , 'I')  :       ['II'], 
    ('lnl'  , 'II') :       ['III'], 
    ('lnl'  , 'III'):       ['IV'], 
    ('lnl'  , 'IV') :       []
}

Every key in this dictionary is a tuple of the form `({type}, {name})` and represents either a tumor - in which case the `{type}` must be `'tumor'` - or a lymph node level (`{type}` must be `'lnl'`). The value of each of those nodes is then a list of names for nodes it connects to. So, for example the primary tumor `('tumor', 'primary')` in the `graph` above has directed arcs to `a` and `b`, while the LNL `c` does not have any outgoing connections.

We can simply create an instance of `System` using only this graph and let it report itself:

In [ ]:
model = lymph.models.Unilateral.binary(graph_dict=graph_dict)
model

The percentages between two nodes represents the probability rate that metastatic spread occurs along it. In the case of the tumor spreading to LNL `II` we call this probability *base probability rate* $\tilde{b}_2$. For the spread between lymph node levels, we call it *transition probability rate*, e.g. $\tilde{t}_{rv}$ from LNL $r$ to LNL $v$. The difference to the base probability rate is that it only plays a role if the parent LNL is already ivolved with metastases, while the tumor always spreads, of course.

We can change these spread probability rates either via the lookup dictionary `edge_params` of the model's `graph` attribute, or directly via the method `assign_parms()`, which can also handle parameters not related to the spread probability rates of the arcs between LNLs. In the former case, the keys to set the spread probabilities are built like `"spread_" + parent.name + "_to_" + child.name`.

In [ ]:
model.graph.edges["T_to_II"].set_params(spread=0.5)
model.graph.edges["T_to_II"].get_params("spread")

In [ ]:
model.assign_params(II_to_III_spread=0.25)
model.get_params(as_dict=True)

## Diagnostic Modalities

To ultimately compute the likelihoods of observations, we need to fix the sensitivities and specificities of the obtained diagnoses. And since we might have multiple diagnostic modalities available, we need to tell the system which of them comes with which specificity and sensitivity. We do this by creating a dictionary of specificity/sensitivity pairs:

In [ ]:
mri_and_pet_spsn = {"MRI": [0.63, 0.81], 
                    "PET": [0.86, 0.79]}
#                           ^     ^
#                  specificty     sensitivity

Now we can pass this to the system by setting the `modalities` attribute, which expects a dictionary containing the diagnostic modalities and as a key to it the numbers for specificity & sensitivity.

In [ ]:
model.modalities = mri_and_pet_spsn
model.modalities

One can also specify the values of the modality dictionary with the helper classes `Clincal` and `Pathological`. In the binary model case that case no advantage over the method above, aside from being maybe a bit more readable. But when we get to trinary models, it becomes very important.

In [ ]:
from lymph.modalities import Clinical

model.modalities["CT"] = Clinical(specificity=0.86, sensitivity=0.81)
model.modalities

Now it is also possible to access the confusion matrix of the specified diagnostic modalities:

In [ ]:
print(model.modalities["PET"].confusion_matrix)

## Data / Observations

To compute the likelihood of a set of probability (rates) given a patient cohort we need such a patient cohort, of course. We can provide it to the system in the form of a `pandas` `DataFrame`. Here is an example:

In [ ]:
import pandas as pd

dataset_url = "https://raw.githubusercontent.com/rmnldwg/lydata/main/2021-usz-oropharynx/data.csv"
example_cols = [
    ("patient", "#", "age"),
    ("patient", "#", "hpv_status"),
    ("tumor", "1", "t_stage"),
    ("PET", "ipsi", "I"),
    ("PET", "ipsi", "II"),
    ("PET", "ipsi", "III"),
    ("PET", "ipsi", "IV"),
    ("MRI", "ipsi", "I"),
    ("MRI", "ipsi", "II"),
    ("MRI", "ipsi", "III"),
    ("MRI", "ipsi", "IV"),
]

usz_oropharynx = pd.read_csv(dataset_url, header=[0,1,2])
usz_oropharynx[example_cols]

:::{note}

This data has three header-rows. It follows the standard of the data presented in the interactive dashboard [LyProX]. It can be obtained from the [lyDATA] repository on GitHub and is described in more detail [here].
:::

[LyProX]: https://lyprox.org
[lyDATA]: https://github.com/rmnldwg/lydata
[here]: https://github.com/rmnldwg/lydata/tree/main/2021-usz-oropharynx

Let's keep only the `"CT"` modality for now.

In [ ]:
model.modalities = {"PET": [0.86, 0.79]}

To feed the dataset into the system, we assign the dataset to the attribute `patient_data`. What the system then does here is creating a diagnose matrix for every T-stage in the data.

In [ ]:
model.load_patient_data(usz_oropharynx, side="ipsi")
model.patient_data["_model"]

:::{note}

The data now has an additional top-level header `"_model"` which stores only the information the model actually needs. In this case, it only stores the ipsilateral CT diagnoses of the LNLs I, II, III, and IV, as well as the mapped T-stage of the patients. Note that from the original T-stages 1, 2, 3, and 4, only "early" and "late" are left. This is the default transformation, but it can be changed by providing a function to the `mapping` keyword argument in the `load_patient_data()` method.
:::

## Distribution over Diagnose Times

The last ingredient to set up (at least when using the hidden Markov model) would now be the distribution over diagnose times. Our dataset contains two different T-stages "early" and "late". One of the underlying assumptions with our model is that earlier T-stage patients have been - on average - diagnosed at an earlier time-point, compared to late T-stage patients. We can reflect that using distributions over the diagnosis time:

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

max_time = model.max_time
time_steps = np.arange(max_time+1)
p = 0.4

early_prior = sp.stats.binom.pmf(time_steps, max_time, p)

plt.plot(time_steps, early_prior, "o-");
plt.xlabel("time step $t$");
plt.ylabel("probability $p$");

We can now set a fixed prior for the distribution over diagnose times of early T-stage patients (i.e., patients with T1 and T2 tumors).

In [ ]:
model.diag_time_dists["early"] = early_prior

Let's define a parametrized PMF over diagnose times for patients with late T-stage tumors (T3 and T4) to show this functionality. For that, we first define a parametrized function with the signature

```python
def distribution(support: list[float] | np.ndarray, a=1, b=2, c=3, ...) -> np.ndarray:
    """PMF over diagnose times (``support``) with parameters ``a``, ``b``, and ``c``."""
    ...
    return result
```

Here, it's important that the first argument is the support of the probability mass function, i.e., the discrete time-steps from 0 to `max_time`. Also, all parameters must have default values. Otherwise, there would be cases when such a stored distribution cannot be accessed.

Lastly, if some parameters have bounds, like e.g. the binomial distribution, they should raise a `ValueError`. This exception is propagated upwards but causes the `likelihood` method to simply return `-np.inf`. That way it will be seamlessly rejected during an MCMC sampling round.

Let's look at a concrete, binomial example:

In [ ]:
from scipy.special import factorial

def binom_pmf(k: np.ndarray, n: int, p: float):
    """Binomial PMF"""
    if p > 1. or p < 0.:
        # This value error is important to enable seamless sampling!
        raise ValueError("Binomial prob must be btw. 0 and 1")
    q = (1. - p)
    binom_coeff = factorial(n) / (factorial(k) * factorial(n - k))
    return binom_coeff * p**k * q**(n - k)

def late_binomial(support: np.ndarray, p: float = 0.5) -> np.ndarray:
    """Parametrized binomial distribution."""
    return binom_pmf(k=support, n=support[-1], p=p)

:::{note}

Surprisingly, this is much slower than just implementing it "by hand" using numpy array functions. So, if you want to do sampling with a model, maybe don't use `scipy.stats`.
:::

And now we assign this parametric distribution to the model:

In [ ]:
model.diag_time_dists["late"] = late_binomial
model.get_params(as_dict=True)

Note how the set of adjustable parameters now also contains the `p` parameter for the late T-stage's distribution over diagnose times. For the early T-stage, it is not present, because that one was provided as a fixed array.

## Likelihood

With everything set up like this, we can compute the likelihood of seeing the above dataset given a set of base and transition probability (rates).

In [ ]:
test_probabilities = [0.02, 0.24, 0.03, 0.2, 0.23, 0.18, 0.18, 0.5]

llh = model.likelihood(given_param_args=test_probabilities, log=True)

print(f"log-likelihood is {llh}")

From here it is up to the user what to do with this quantity. Most *likely* though, one would want to perform MCMC sampling with this.

## Summary

To set up a model for lymphatic metastatic spread, you need to do the following things:

1. Define a graph via a dictionary that connects the tumor and lymph node levels
2. Provide the specificity & sensitivity of the diagnostic modalities to the `modalities` attribute
3. Assign your correctly formatted pandas `DataFrame` to the method `load_patient_data()` of the model
4. For each T-stage in the data, define a distribution over possible diagnose times and provide it to the `diag_time_dist` dictionary

Then, you can use the `likelihood` method to compute the (log-)likelihood of the stored data given an array of parameters - the spread probabilities.